# **Genomics Data Analysis: ONT**

Introductory 
(*under dev...*)

### **Setup**
Assumes a Linux system running Jupyter

***Install...***
- Guppy
- FastQC
- [Minimap2](https://github.com/lh3/minimap2)
- Samtools
- [Medaka](https://github.com/nanoporetech/medaka)
- [snpEff](http://pcingola.github.io/SnpEff/download/)

### **Organisation**
- Input data directory
- Analyses output directory
- Logs directory*

### **Fast5 file format**
<img src="https://github.com/ousodaniel/Images/blob/master/HyperLinkedImages/hd5_format.jpg" alt="Hierarchical Data Formats" style="width: 200"/>

Photo from [neonscience](https://www.neonscience.org/resources/learning-hub/tutorials/about-hdf5)

### **Analysis**

In [ ]:
!guppy_basecaller \
    --disable_pings \
    --device "auto" \
    --config dna_r9.4.1_450bps_fast.cfg \  # alt --kit --flowcell
    --min_score 9 \
#        
    --recursive \  # careful with this
    --input_path $input_dir \
    --compress_fastq \
    --fast5_out \
    --save_path $out_dir
#
# runners * chunks_per_runner * chunk_size < 100000 * [max GPU memory in Gb]
    --num_callers 1 \
    --gpu_runners_per_device 5 \
    --chunks_per_runner 512 \
    --chunk_size 100  # assuming 4Gb ram
#    --resume


In [ ]:
!guppy_barcoder \
    --disable_pings \
    --device "auto" \
#
    --barcode_kits "EXP-NBD196 EXP-NBD104 EXP-NBD114 SQK-NBD110-24 SQK-NBD110-96" \  # choose which aplies
    --min_qscore 60 \
    --allow_inferior_barcodes \
    --detect_mid_strand_barcodes \
    --require_barcodes_both_ends \
    --arrangements_files samplesheet.csv \
    --trim_barcodes \
#        
    --recursive \
    --input_path $input_dir \
    --compress_fastq \
    --save_path $out_dir 

In [ ]:
!fastqc -o $output_dir -f $reads_fastq

In [ ]:
!minimap2 -ax sr $ref_fasta $read1_fastq $read2_fastq > $aln_sam

In [ ]:
!samtools view -bS $aln_sam > $aln_bam

In [ ]:
!medaka_consensus -t 2 \  # number of threads
 -d $ref_fasta \
 -i $reads_fastq \
 -o $output_dir \
 -m r941_min_fast_g303  # model choice: based on flowcell, machine and basecalling mode

In [ ]:
!medaka_variant -f $ref_fasta -i $input_bam -o $output_dir

In [ ]:
# see availavle databases
!java -jar snpEff.jar databases  # database name, genome name and source data

In [ ]:
# annotate
!java -Xmx2g -Xmx4g -jar snpEff.jar -c snpEff.config \
$database_name $variants_vcf > $variants_ann_vcf